## **Import the libraries** ##

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

## **Create the files** ##
Dont need to run this part again

In [ ]:
#Create the dataframes
df_clients = pd.read_csv("../Data/Raw/df_final_experiment_clients.txt")
df_demo = pd.read_csv("../Data/Raw/df_final_demo.txt")
df_web1 = pd.read_csv("../Data/Raw/df_final_web_data_pt_1.txt")
df_web2 = pd.read_csv("../Data/Raw/df_final_web_data_pt_2.txt")
df_clients.head()

In [ ]:
#Concat data_web
df_web=pd.concat([df_web1,df_web2], axis=0)
df_web=df_web.reset_index(drop=True)
df_web

In [ ]:
#Merge demo with clients
df_demo=pd.merge(df_demo,df_clients, on="client_id", how="left")



In [ ]:
#I create demo_control
conditioncontrol=df_demo["Variation"]=="Control"
df_demo_control=df_demo[conditioncontrol]
df_demo_control

In [ ]:
#I create demo_test
conditiontest=df_demo["Variation"]=="Test"
df_demo_test=df_demo[conditiontest]
df_demo_test

In [ ]:
#I add information about the clientes to data_web
df_web=pd.merge(df_web,df_clients, on="client_id", how="left")
df_web

In [ ]:
#Create web_test
conditiontest=df_web["Variation"]=="Test"
df_web_test=df_web[conditiontest]
df_web_test

In [ ]:
#Create web_control
conditioncontrol=df_web["Variation"]=="Control"
df_web_control=df_web[conditioncontrol]
df_web_control

In [ ]:
#FINAL RESULT 4 FILES:
#df_web_control
#df_web_test
#df_demo_control
#df_demo_test
df_web_test.to_csv("../Data/Raw/df_final_web_data_test.txt")
df_web_control.to_csv("../Data/Raw/df_final_web_data_control.txt")

## **Cleaning and columns aggregation** ##

In [ ]:
#Clean web_data_test 1148 duplicated values
df_web_test = pd.read_csv("../Data/Raw/df_final_web_data_test.txt")
df_web_test=df_web_test.reset_index(drop=True)
df_web_test.duplicated().sum()
df_web_test.drop_duplicates(inplace=True)
df_web_test.duplicated().sum()

In [ ]:
#Clean web_data_test 1148 duplicated values
df_web_control = pd.read_csv("../Data/Raw/df_final_web_data_control.txt")
df_web_control=df_web_control.reset_index(drop=True)
df_web_control.duplicated().sum()
df_web_control.drop_duplicates(inplace=True)
df_web_control.duplicated().sum()

In [ ]:
#Split date and time
df_web_test["date"]=df_web_test["date_time"].str.split(" ", expand=True)[0]
df_web_test["time"]=df_web_test["date_time"].str.split(" ", expand=True)[1]
df_web_test["hour"]=df_web_test["time"].str.split(":",expand=True)[0]
df_web_test["hour"]=df_web_test["hour"].astype(int)
df_web_test.head()

In [ ]:
#Split date and time
df_web_control["date"]=df_web_control["date_time"].str.split(" ", expand=True)[0]
df_web_control["time"]=df_web_control["date_time"].str.split(" ", expand=True)[1]
df_web_control["hour"]=df_web_control["time"].str.split(":",expand=True)[0]
df_web_control["hour"]=df_web_control["hour"].astype(int)
df_web_control.head()

In [ ]:
#Change the dtypes of the column time %H,%M,%S format
df_web_test['time'] = (pd.to_datetime(df_web_test['time'], format='%H:%M:%S'))
df_web_test

In [ ]:
#Change the dtypes of the column time %H,%M,%S format
df_web_control['time'] = (pd.to_datetime(df_web_control['time'], format='%H:%M:%S'))
df_web_control

In [ ]:
#Change the values of process_step
df_web_test["process_step"]=df_web_test["process_step"].map({"start":0,"step_1":1,"step_2":2,"step_3":3,"confirm":4})
df_web_test["process_step"].unique()

In [ ]:
#Change the values of process_step
df_web_control["process_step"]=df_web_control["process_step"].map({"start":0,"step_1":1,"step_2":2,"step_3":3,"confirm":4})
df_web_control["process_step"].unique()

In [ ]:
#Create two new columns in order to have the step_jump and one column "control" to filter when the data is not from the same visit_id.
df_web_test=df_web_test.sort_values(["visit_id","date", "time", "process_step"])
df_web_test["step_jump"]=df_web_test["process_step"].astype(str).shift(+1)+"-"+df_web_test["process_step"].astype(str)
#Create column control which help me to determine, later, is the steps follows a orded sense.
df_web_test["control"]=(df_web_test["process_step"]-df_web_test["process_step"].shift(+1)).abs()
df_web_test["dif_time"]=(df_web_test["time"])-(df_web_test["time"].shift(+1))
df_web_test

In [ ]:
#Create two new columns in order to have the step_jump and one column "control" to filter when the data is not from the same visit_id.
df_web_control=df_web_control.sort_values(["visit_id","date", "time", "process_step"])
df_web_control["step_jump"]=df_web_control["process_step"].astype(str).shift(+1)+"-"+df_web_control["process_step"].astype(str)
#Create column control which help me to determine, later, is the steps follows a orded sense.
df_web_control["control"]=(df_web_control["process_step"]-df_web_control["process_step"].shift(+1)).abs()
df_web_control["dif_time"]=(df_web_control["time"])-(df_web_control["time"].shift(+1))
df_web_control

In [ ]:
#Create a column for the total of segs spend between steps
df_web_test=df_web_test.dropna(subset=["control"])
df_web_test["dif_time"] = (df_web_test['dif_time'].astype(str))
df_web_test["dif_time"]=(df_web_test["dif_time"].str.split(" ")).str[-1]
df_web_test["dif_hour"]=((df_web_test["dif_time"].str.split(":")).str[0])
df_web_test["dif_hour"]=df_web_test["dif_hour"].astype(int)
df_web_test["dif_min"]=((df_web_test["dif_time"].str.split(":")).str[1]).astype(int)
df_web_test["dif_seg"]=((df_web_test["dif_time"].str.split(":")).str[2]).astype(int)
df_web_test["total_seg"]=df_web_test["dif_hour"]*3600 + df_web_test["dif_min"]*60 + df_web_test["dif_seg"]
df_web_test

In [ ]:
#Create a column for the total of segs spend between steps
df_web_control=df_web_control.dropna(subset=["control"])
df_web_control["dif_time"] = (df_web_control['dif_time'].astype(str))
df_web_control["dif_time"]=(df_web_control["dif_time"].str.split(" ")).str[-1]
df_web_control["dif_hour"]=((df_web_control["dif_time"].str.split(":")).str[0])
df_web_control["dif_hour"]=df_web_control["dif_hour"].astype(int)
df_web_control["dif_min"]=((df_web_control["dif_time"].str.split(":")).str[1]).astype(int)
df_web_control["dif_seg"]=((df_web_control["dif_time"].str.split(":")).str[2]).astype(int)
df_web_control["total_seg"]=df_web_control["dif_hour"]*3600 + df_web_control["dif_min"]*60 + df_web_control["dif_seg"]
df_web_control

In [ ]:
#Function for the keep_control column. It is only going to be used in terms of TIME BETWEEN STEPS
def row_count(df):
    df['next_visit'] = df['visit_id'].shift(+1) 
    df['keep_contol'] = df.apply(lambda row: "YES" if row['visit_id'] == row['next_visit'] else "NO", axis=1)
    df.drop(columns=['next_visit'], inplace=True)
    return df

In [ ]:
#Function for the keep_control column. It is only going to be used in terms of TIME BETWEEN STEPS
def row_count(df):
    df['next_visit'] = df['visit_id'].shift(+1) 
    df['keep_contol'] = df.apply(lambda row: "YES" if row['visit_id'] == row['next_visit'] else "NO", axis=1)
    df.drop(columns=['next_visit'], inplace=True)
    return df

In [ ]:
#Add the column keep_control to my DataFrame
df_web_test=row_count(df_web_test)
df_web_test
df_web_test.to_csv("../Data/Clean/df_final_web_data_test_clean.txt")
#FINAL DATA FRAME TO USE: df_web_test

In [ ]:
#Add the column keep_control to my DataFrame
df_web_control=row_count(df_web_control)
df_web_control
df_web_control.to_csv("../Data/Clean/df_final_web_data_control_clean.txt")
#FINAL DATA FRAME TO USE: df_web_control